SCRIPT TO TRAIN W2V/TFIDF ON TOKENIZED TEXT - code from aurelie

In [1]:
# import packages required

import pandas as pd
import os
import numpy as np
from gensim.models import Word2Vec, FastText
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim import corpora
from gensim.models import TfidfModel
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
# spacy - skip - this outputs an error

if os.getcwd() == '/content': # on google drive
    import spacy
    nlp = spacy.load('en')
else: # on CRIS azure
    import scispacy
    import spacy
    nlp = spacy.load('en_core_sci_md')
    

In [5]:
!pip install scispacy

You should consider upgrading via the '/home/ubuntu/anaconda3/bin/python -m pip install --upgrade pip' command.


In [8]:
! pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_md-0.4.0.tar.gz

     |████████████████████████████████| 125.2 MB 108 kB/s eta 0:00:01    |███████▌                        | 29.3 MB 211 kB/s eta 0:07:34     |███████▌                        | 29.5 MB 230 kB/s eta 0:06:56     |████████████▍                   | 48.5 MB 284 kB/s eta 0:04:31     |█████████████▋                  | 53.2 MB 172 kB/s eta 0:06:57��██              | 70.7 MB 709 kB/s eta 0:01:17
  Created wheel for en-core-sci-md: filename=en_core_sci_md-0.4.0-py3-none-any.whl size=125733930 sha256=19855a88206d123b61748d5162a699ed089e3b6260b409fa8e9520a7c2a819e6
  Stored in directory: /home/ubuntu/.cache/pip/wheels/55/c0/ee/f9f0abbcbe0bd9da49c05b04aba66f0d259dd6af60b207c9d3
Successfully built en-core-sci-md
  Attempting uninstall: en-core-sci-md
    Found existing installation: en-core-sci-md 0.3.0
    Uninstalling en-core-sci-md-0.3.0:
      Successfully uninstalled en-core-sci-md-0.3.0
You should consider upgrading via the '/home/ubuntu/anaconda3/bin/python -m pip install --upgrade pip' comman

In [9]:
# spacy
import scispacy
import spacy
#nlp = spacy.load('en')
nlp = spacy.load('en_core_sci_md')

In [10]:
from spacy.lang.en.examples import sentences

## PREPARE TEXT FILE

In [18]:
def tokenize_spacy(df, text_col=None, tokenization_type='clean', outfile=None):
    tok_snts = []
    if outfile is not None: f = open(outfile, 'w', encoding='utf8')
    data = df if text_col is None else df[text_col]
    for snt in data:
        tkns = nlp.tokenizer(snt)
        if ('low' in tokenization_type) and ('wos' in tokenization_type):
            _tkns = [str(x.text).lower() for x in tkns if not x.is_space]
        elif 'wos' in tokenization_type:
            _tkns = [str(x.text) for x in tkns if not x.is_space]
        elif 'lem' in tokenization_type:
            _tkns = [str(x.lemma_).lower() for x in tkns if not x.is_space and not x.is_punct]
        elif 'stem' in tokenization_type:
            _tkns = [stemmer.stem(str(x.text).lower()) for x in tkns if not x.is_space and not x.is_punct]
        else: # clean by default
            _tkns = [str(x.text).lower() for x in tkns if not x.is_space and not x.is_punct]
        
        if outfile is not None: # flush to file if option selected
            f.write("{}\n".format("\t".join(_tkns)))
        else: # otherwise save in variable
            tok_snts.append(_tkns)

    return tok_snts if outfile is None else None


In [19]:
# PREPARE THE TXT FILE
#raw_file = '/home/ubuntu/data/mimic/tokenized_scispacy/noteevents_tkns.csv
raw_file = './NOTEEVENTS_all_text.csv'

In [20]:
df = pd.read_csv(raw_file)

In [21]:
df.head()

,text
0,Coronary artery bypass graft (CABG)\n Assess...
1,Subdural hemorrhage (SDH)\n Assessment:\n ...
2,Coronary artery bypass graft (CABG)\n Assess...
3,Coronary artery bypass graft (CABG)\n Assess...
4,Demographics\n Day of intubation:\n Day of...


In [25]:
# this step runs but the outfile seems to be empty? is it because text_col should be 'text'? this seems to have worked


#sentences = tokenize_spacy(df, text_col=None, tokenization_type='clean', outfile=raw_file,replace('.csv','_clean.txt'))
sentences = tokenize_spacy(df, text_col='text', tokenization_type='clean', outfile='./notes_clean.txt')

In [32]:
#numpy_array = df.to_numpy()

In [ ]:
#np.savetxt("noteevents_v2.txt", numpy_array, fmt='%s')

## INPUT DATA

In [37]:
# INPUT PARAMETERS

# tokenized text
#filename = '/home/ubuntu/data/mimic/tokenized_scispacy/noteevents_tkns_stem.txt' # with stemming
filename = './notes_clean.txt'

# w2v/fasttext params
size=300
window=10
min_count=5
workers=16

# sklearn - tfidf params
min_df=0.2
max_df=0.9
ngram_range=(1, 5)
max_features=1000

# gensim - tfidf params

In [38]:
class SentenceIterator: 
    def __init__(self, filepath): 
        self.filepath = filepath 

    def __iter__(self): 
        for line in open(self.filepath): 
            yield line.split("\t") 

In [39]:
sentences = SentenceIterator(filename) 

## TRAIN GLOVE

In [ ]:
!pip install glove # this is not working.. have to figure out how to make glove work here - or ask tao

In [31]:
# TRAIN AND SAVE GLOVE
import glove
from glove import Corpus, Glove


corpus = Corpus() #Creating a corpus object
corpus.fit(sentences, window=window) #Training the corpus to generate the co occurence matrix which is used in GloVe

glove = Glove(no_components=5, learning_rate=0.05) 
glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
glove.save(filename.replace('.txt','_glove.model').replace('_tkns',''))

ModuleNotFoundError: No module named 'glove'

## TRAIN FASTEXT

In [40]:
# TRAIN AND SAVE FASTTEXT
ft_model = FastText(size=size, window=window, min_count=min_count)  # instantiate
ft_model.build_vocab(sentences=sentences)
ft_model.train(sentences=sentences, total_examples=sum(1 for _ in sentences), epochs=2)
#ft_model.save(filename.replace('.txt','_fasttext_win10_min5.dat').replace('_tkns',''))
ft_model.save(filename.replace('.txt','_fasttext_win10_min5_v2.dat'))

2021-02-23 17:03:53,717 : INFO : resetting layer weights
2021-02-23 17:04:09,017 : INFO : collecting all words and their counts
2021-02-23 17:04:09,019 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-02-23 17:04:10,177 : INFO : PROGRESS: at sentence #10000, processed 4750988 words, keeping 47253 word types
2021-02-23 17:04:11,350 : INFO : PROGRESS: at sentence #20000, processed 9517988 words, keeping 70404 word types
2021-02-23 17:04:12,516 : INFO : PROGRESS: at sentence #30000, processed 14217892 words, keeping 89904 word types
2021-02-23 17:04:13,270 : INFO : PROGRESS: at sentence #40000, processed 17342834 words, keeping 108098 word types
2021-02-23 17:04:13,746 : INFO : PROGRESS: at sentence #50000, processed 19317065 words, keeping 123739 word types
2021-02-23 17:04:14,224 : INFO : PROGRESS: at sentence #60000, processed 21292462 words, keeping 137319 word types
2021-02-23 17:04:14,719 : INFO : PROGRESS: at sentence #70000, processed 23293819 words,

2021-02-23 17:04:42,091 : INFO : PROGRESS: at sentence #700000, processed 128065898 words, keeping 685141 word types
2021-02-23 17:04:42,467 : INFO : PROGRESS: at sentence #710000, processed 129482719 words, keeping 690729 word types
2021-02-23 17:04:42,959 : INFO : PROGRESS: at sentence #720000, processed 131326251 words, keeping 698923 word types
2021-02-23 17:04:43,539 : INFO : PROGRESS: at sentence #730000, processed 133405061 words, keeping 707962 word types
2021-02-23 17:04:46,813 : INFO : PROGRESS: at sentence #740000, processed 146962855 words, keeping 765561 word types
2021-02-23 17:04:50,586 : INFO : PROGRESS: at sentence #750000, processed 162247244 words, keeping 810963 word types
2021-02-23 17:04:54,484 : INFO : PROGRESS: at sentence #760000, processed 178597370 words, keeping 851152 word types
2021-02-23 17:04:55,299 : INFO : PROGRESS: at sentence #770000, processed 181862498 words, keeping 853644 word types
2021-02-23 17:04:56,114 : INFO : PROGRESS: at sentence #780000, 

2021-02-23 17:05:37,158 : INFO : PROGRESS: at sentence #1400000, processed 341505124 words, keeping 1173366 word types
2021-02-23 17:05:39,789 : INFO : PROGRESS: at sentence #1410000, processed 352027794 words, keeping 1196703 word types
2021-02-23 17:05:43,500 : INFO : PROGRESS: at sentence #1420000, processed 365689697 words, keeping 1223064 word types
2021-02-23 17:05:44,306 : INFO : PROGRESS: at sentence #1430000, processed 368701155 words, keeping 1223883 word types
2021-02-23 17:05:44,485 : INFO : PROGRESS: at sentence #1440000, processed 369401663 words, keeping 1224097 word types
2021-02-23 17:05:44,564 : INFO : PROGRESS: at sentence #1450000, processed 369708850 words, keeping 1224238 word types
2021-02-23 17:05:44,642 : INFO : PROGRESS: at sentence #1460000, processed 370007394 words, keeping 1224371 word types
2021-02-23 17:05:44,722 : INFO : PROGRESS: at sentence #1470000, processed 370318348 words, keeping 1224472 word types
2021-02-23 17:05:44,817 : INFO : PROGRESS: at se

2021-02-23 17:06:28,169 : INFO : collected 1492743 word types from a corpus of 534796798 raw words and 2083180 sentences
2021-02-23 17:06:28,169 : INFO : Loading a fresh vocabulary
2021-02-23 17:06:29,998 : INFO : effective_min_count=5 retains 416342 unique words (27% of original 1492743, drops 1076401)
2021-02-23 17:06:29,999 : INFO : effective_min_count=5 leaves 533154263 word corpus (99% of original 534796798, drops 1642535)
2021-02-23 17:06:31,229 : INFO : deleting the raw counts dictionary of 1492743 items
2021-02-23 17:06:31,273 : INFO : sample=0.001 downsamples 35 most-common words
2021-02-23 17:06:31,274 : INFO : downsampling leaves estimated 476070309 word corpus (89.3% of prior 533154263)
2021-02-23 17:06:36,963 : INFO : estimated required memory for 416342 words, 1062221 buckets and 300 dimensions: 2583428840 bytes
2021-02-23 17:06:37,023 : INFO : resetting layer weights
2021-02-23 17:08:50,243 : INFO : training model with 3 workers on 416342 vocabulary and 300 features, usi

2021-02-23 17:10:02,447 : INFO : EPOCH 1 - PROGRESS: at 0.75% examples, 94766 words/s, in_qsize 5, out_qsize 0
2021-02-23 17:10:03,548 : INFO : EPOCH 1 - PROGRESS: at 0.76% examples, 94774 words/s, in_qsize 5, out_qsize 0
2021-02-23 17:10:04,626 : INFO : EPOCH 1 - PROGRESS: at 0.78% examples, 94805 words/s, in_qsize 5, out_qsize 0
2021-02-23 17:10:05,684 : INFO : EPOCH 1 - PROGRESS: at 0.79% examples, 94862 words/s, in_qsize 5, out_qsize 0
2021-02-23 17:10:06,772 : INFO : EPOCH 1 - PROGRESS: at 0.80% examples, 94892 words/s, in_qsize 5, out_qsize 0
2021-02-23 17:10:07,858 : INFO : EPOCH 1 - PROGRESS: at 0.81% examples, 94935 words/s, in_qsize 5, out_qsize 0
2021-02-23 17:10:08,931 : INFO : EPOCH 1 - PROGRESS: at 0.82% examples, 94985 words/s, in_qsize 5, out_qsize 0
2021-02-23 17:10:10,031 : INFO : EPOCH 1 - PROGRESS: at 0.84% examples, 94963 words/s, in_qsize 5, out_qsize 0
2021-02-23 17:10:11,134 : INFO : EPOCH 1 - PROGRESS: at 0.85% examples, 94961 words/s, in_qsize 5, out_qsize 0
2

...

2021-02-23 19:56:00,268 : INFO : EPOCH 2 - PROGRESS: at 94.77% examples, 92194 words/s, in_qsize 5, out_qsize 0
2021-02-23 19:56:01,430 : INFO : EPOCH 2 - PROGRESS: at 94.77% examples, 92191 words/s, in_qsize 4, out_qsize 1
2021-02-23 19:56:02,478 : INFO : EPOCH 2 - PROGRESS: at 94.78% examples, 92192 words/s, in_qsize 5, out_qsize 0
2021-02-23 19:56:03,525 : INFO : EPOCH 2 - PROGRESS: at 94.78% examples, 92192 words/s, in_qsize 5, out_qsize 0
2021-02-23 19:56:04,634 : INFO : EPOCH 2 - PROGRESS: at 94.78% examples, 92190 words/s, in_qsize 5, out_qsize 0
2021-02-23 19:56:05,757 : INFO : EPOCH 2 - PROGRESS: at 94.79% examples, 92189 words/s, in_qsize 5, out_qsize 0
2021-02-23 19:56:06,906 : INFO : EPOCH 2 - PROGRESS: at 94.79% examples, 92188 words/s, in_qsize 5, out_qsize 0
2021-02-23 19:56:08,002 : INFO : EPOCH 2 - PROGRESS: at 94.79% examples, 92187 words/s, in_qsize 5, out_qsize 0
2021-02-23 19:56:09,003 : INFO : EPOCH 2 - PROGRESS: at 94.80% examples, 92186 words/s, in_qsize 6, out_

2021-02-23 19:57:18,806 : INFO : EPOCH 2 - PROGRESS: at 95.29% examples, 92068 words/s, in_qsize 5, out_qsize 0
2021-02-23 19:57:19,839 : INFO : EPOCH 2 - PROGRESS: at 95.30% examples, 92064 words/s, in_qsize 5, out_qsize 0
2021-02-23 19:57:20,882 : INFO : EPOCH 2 - PROGRESS: at 95.31% examples, 92061 words/s, in_qsize 5, out_qsize 0
2021-02-23 19:57:21,921 : INFO : EPOCH 2 - PROGRESS: at 95.32% examples, 92057 words/s, in_qsize 5, out_qsize 0
2021-02-23 19:57:22,982 : INFO : EPOCH 2 - PROGRESS: at 95.33% examples, 92054 words/s, in_qsize 6, out_qsize 1
2021-02-23 19:57:23,999 : INFO : EPOCH 2 - PROGRESS: at 95.35% examples, 92052 words/s, in_qsize 5, out_qsize 0
2021-02-23 19:57:25,004 : INFO : EPOCH 2 - PROGRESS: at 95.36% examples, 92048 words/s, in_qsize 5, out_qsize 0
2021-02-23 19:57:26,011 : INFO : EPOCH 2 - PROGRESS: at 95.37% examples, 92045 words/s, in_qsize 5, out_qsize 0
2021-02-23 19:57:27,051 : INFO : EPOCH 2 - PROGRESS: at 95.38% examples, 92042 words/s, in_qsize 5, out_

2021-02-23 19:58:38,414 : INFO : EPOCH 2 - PROGRESS: at 98.28% examples, 92009 words/s, in_qsize 6, out_qsize 0
2021-02-23 19:58:39,454 : INFO : EPOCH 2 - PROGRESS: at 98.29% examples, 92011 words/s, in_qsize 5, out_qsize 0
2021-02-23 19:58:40,460 : INFO : EPOCH 2 - PROGRESS: at 98.30% examples, 92012 words/s, in_qsize 5, out_qsize 0
2021-02-23 19:58:41,549 : INFO : EPOCH 2 - PROGRESS: at 98.31% examples, 92012 words/s, in_qsize 5, out_qsize 0
2021-02-23 19:58:42,628 : INFO : EPOCH 2 - PROGRESS: at 98.32% examples, 92013 words/s, in_qsize 5, out_qsize 0
2021-02-23 19:58:43,713 : INFO : EPOCH 2 - PROGRESS: at 98.34% examples, 92014 words/s, in_qsize 5, out_qsize 0
2021-02-23 19:58:44,763 : INFO : EPOCH 2 - PROGRESS: at 98.35% examples, 92015 words/s, in_qsize 5, out_qsize 0
2021-02-23 19:58:45,845 : INFO : EPOCH 2 - PROGRESS: at 98.36% examples, 92017 words/s, in_qsize 5, out_qsize 0
2021-02-23 19:58:46,932 : INFO : EPOCH 2 - PROGRESS: at 98.38% examples, 92018 words/s, in_qsize 5, out_

2021-02-23 19:59:58,394 : INFO : EPOCH 2 - PROGRESS: at 99.18% examples, 92070 words/s, in_qsize 6, out_qsize 0
2021-02-23 19:59:59,453 : INFO : EPOCH 2 - PROGRESS: at 99.19% examples, 92070 words/s, in_qsize 5, out_qsize 0
2021-02-23 20:00:00,563 : INFO : EPOCH 2 - PROGRESS: at 99.20% examples, 92071 words/s, in_qsize 5, out_qsize 0
2021-02-23 20:00:01,673 : INFO : EPOCH 2 - PROGRESS: at 99.21% examples, 92071 words/s, in_qsize 5, out_qsize 0
2021-02-23 20:00:02,804 : INFO : EPOCH 2 - PROGRESS: at 99.23% examples, 92071 words/s, in_qsize 4, out_qsize 1
2021-02-23 20:00:03,947 : INFO : EPOCH 2 - PROGRESS: at 99.24% examples, 92071 words/s, in_qsize 5, out_qsize 0
2021-02-23 20:00:05,061 : INFO : EPOCH 2 - PROGRESS: at 99.25% examples, 92071 words/s, in_qsize 5, out_qsize 0
2021-02-23 20:00:06,163 : INFO : EPOCH 2 - PROGRESS: at 99.26% examples, 92072 words/s, in_qsize 5, out_qsize 0
2021-02-23 20:00:07,278 : INFO : EPOCH 2 - PROGRESS: at 99.27% examples, 92073 words/s, in_qsize 5, out_

2021-02-23 20:01:32,285 : INFO : storing np array 'vectors_vocab' to ./notes_clean_fasttext_win10_min5_v2.dat.wv.vectors_vocab.npy
2021-02-23 20:01:32,631 : INFO : storing np array 'vectors_ngrams' to ./notes_clean_fasttext_win10_min5_v2.dat.wv.vectors_ngrams.npy
2021-02-23 20:01:34,266 : INFO : not storing attribute vectors_norm
2021-02-23 20:01:34,267 : INFO : not storing attribute vectors_vocab_norm
2021-02-23 20:01:34,268 : INFO : not storing attribute vectors_ngrams_norm
2021-02-23 20:01:34,269 : INFO : not storing attribute buckets_word
2021-02-23 20:01:34,270 : INFO : storing np array 'syn1neg' to ./notes_clean_fasttext_win10_min5_v2.dat.trainables.syn1neg.npy
2021-02-23 20:01:34,602 : INFO : storing np array 'vectors_vocab_lockf' to ./notes_clean_fasttext_win10_min5_v2.dat.trainables.vectors_vocab_lockf.npy
2021-02-23 20:01:34,933 : INFO : storing np array 'vectors_ngrams_lockf' to ./notes_clean_fasttext_win10_min5_v2.dat.trainables.vectors_ngrams_lockf.npy
2021-02-23 20:01:37,

first run - using text tokenised and cleaned using nltk:

2021-02-22 10:59:28,104 : INFO : collected 2019963 word types from a corpus of 2083112 raw words and 2083112 sentences
2021-02-22 10:59:28,104 : INFO : Loading a fresh vocabulary
2021-02-22 10:59:28,901 : INFO : effective_min_count=5 retains 1176 unique words (0% of original 2019963, drops 2018787)
2021-02-22 10:59:28,902 : INFO : effective_min_count=5 leaves 23371 word corpus (1% of original 2083112, drops 2059741)
2021-02-22 10:59:28,906 : INFO : deleting the raw counts dictionary of 2019963 items
2021-02-22 10:59:29,173 : INFO : sample=0.001 downsamples 38 most-common words
2021-02-22 10:59:29,174 : INFO : downsampling leaves estimated 15132 word corpus (64.7% of prior 23371)
2021-02-22 10:59:29,656 : INFO : estimated required memory for 1176 words, 78129 buckets and 300 dimensions: 110096912 bytes
2021-02-22 10:59:29,658 : INFO : resetting layer weights
2021-02-22 10:59:45,575 : INFO : training model with 3 workers on 1176 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2021-02-22 11:00:02,367 : INFO : EPOCH - 1 : training on 2083112 raw words (15179 effective words) took 16.8s, 904 effective words/s
2021-02-22 11:00:19,226 : INFO : EPOCH - 2 : training on 2083112 raw words (15148 effective words) took 16.8s, 899 effective words/s
2021-02-22 11:00:19,227 : INFO : training on a 4166224 raw words (30327 effective words) took 33.7s, 901 effective words/s
2021-02-22 11:00:21,558 : INFO : saving FastText object under ./noteevents_clean_fasttext_win10_min5.dat, separately None
2021-02-22 11:00:21,559 : INFO : storing np array 'vectors_ngrams' to ./noteevents_clean_fasttext_win10_min5.dat.wv.vectors_ngrams.npy
2021-02-22 11:00:23,183 : INFO : not storing attribute vectors_norm
2021-02-22 11:00:23,184 : INFO : not storing attribute vectors_vocab_norm
2021-02-22 11:00:23,184 : INFO : not storing attribute vectors_ngrams_norm
2021-02-22 11:00:23,185 : INFO : not storing attribute buckets_word
2021-02-22 11:00:23,186 : INFO : storing np array 'vectors_ngrams_lockf' to ./noteevents_clean_fasttext_win10_min5.dat.trainables.vectors_ngrams_lockf.npy
2021-02-22 11:00:24,810 : INFO : saved ./noteevents_clean_fasttext_win10_min5.dat

second run - using text cleaned with spacy and this script

2021-02-23 17:06:28,169 : INFO : collected 1492743 word types from a corpus of 534796798 raw words and 2083180 sentences
2021-02-23 17:06:28,169 : INFO : Loading a fresh vocabulary
2021-02-23 17:06:29,998 : INFO : effective_min_count=5 retains 416342 unique words (27% of original 1492743, drops 1076401)
2021-02-23 17:06:29,999 : INFO : effective_min_count=5 leaves 533154263 word corpus (99% of original 534796798, drops 1642535)
2021-02-23 17:06:31,229 : INFO : deleting the raw counts dictionary of 1492743 items
2021-02-23 17:06:31,273 : INFO : sample=0.001 downsamples 35 most-common words
2021-02-23 17:06:31,274 : INFO : downsampling leaves estimated 476070309 word corpus (89.3% of prior 533154263)
2021-02-23 17:06:36,963 : INFO : estimated required memory for 416342 words, 1062221 buckets and 300 dimensions: 2583428840 bytes
2021-02-23 17:06:37,023 : INFO : resetting layer weights
2021-02-23 17:08:50,243 : INFO : training model with 3 workers on 416342 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2021-02-23 18:35:02,395 : INFO : EPOCH - 1 : training on 534796798 raw words (476073678 effective words) took 5172.1s, 92046 effective words/s
2021-02-23 20:01:10,385 : INFO : EPOCH - 2 : training on 534796798 raw words (476061984 effective words) took 5168.0s, 92118 effective words/s
2021-02-23 20:01:10,386 : INFO : training on a 1069593596 raw words (952135662 effective words) took 10340.1s, 92081 effective words/s
2021-02-23 20:01:31,925 : INFO : saving FastText object under ./notes_clean_fasttext_win10_min5_v2.dat, separately None
2021-02-23 20:01:31,927 : INFO : storing np array 'vectors' to ./notes_clean_fasttext_win10_min5_v2.dat.wv.vectors.npy
2021-02-23 20:01:32,285 : INFO : storing np array 'vectors_vocab' to ./notes_clean_fasttext_win10_min5_v2.dat.wv.vectors_vocab.npy
2021-02-23 20:01:32,631 : INFO : storing np array 'vectors_ngrams' to ./notes_clean_fasttext_win10_min5_v2.dat.wv.vectors_ngrams.npy
2021-02-23 20:01:34,266 : INFO : not storing attribute vectors_norm
2021-02-23 20:01:34,267 : INFO : not storing attribute vectors_vocab_norm
2021-02-23 20:01:34,268 : INFO : not storing attribute vectors_ngrams_norm
2021-02-23 20:01:34,269 : INFO : not storing attribute buckets_word
2021-02-23 20:01:34,270 : INFO : storing np array 'syn1neg' to ./notes_clean_fasttext_win10_min5_v2.dat.trainables.syn1neg.npy
2021-02-23 20:01:34,602 : INFO : storing np array 'vectors_vocab_lockf' to ./notes_clean_fasttext_win10_min5_v2.dat.trainables.vectors_vocab_lockf.npy
2021-02-23 20:01:34,933 : INFO : storing np array 'vectors_ngrams_lockf' to ./notes_clean_fasttext_win10_min5_v2.dat.trainables.vectors_ngrams_lockf.npy
2021-02-23 20:01:37,458 : INFO : saved ./notes_clean_fasttext_win10_min5_v2.dat

## TRAIN W2v

In [41]:
# TRAIN AND SAVE W2V
w2v_model = Word2Vec(sentences, size=size, window=window, min_count=min_count, workers=workers)
w2v_model.save(filename.replace('.txt','_w2v_win10_min5_v2.dat'))

# test
w2v_model.wv.most_similar("pain")

2021-02-23 20:07:10,224 : INFO : collecting all words and their counts
2021-02-23 20:07:10,228 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-02-23 20:07:11,412 : INFO : PROGRESS: at sentence #10000, processed 4750988 words, keeping 47253 word types
2021-02-23 20:07:12,576 : INFO : PROGRESS: at sentence #20000, processed 9517988 words, keeping 70404 word types
2021-02-23 20:07:13,750 : INFO : PROGRESS: at sentence #30000, processed 14217892 words, keeping 89904 word types
2021-02-23 20:07:14,590 : INFO : PROGRESS: at sentence #40000, processed 17342834 words, keeping 108098 word types
2021-02-23 20:07:15,099 : INFO : PROGRESS: at sentence #50000, processed 19317065 words, keeping 123739 word types
2021-02-23 20:07:15,635 : INFO : PROGRESS: at sentence #60000, processed 21292462 words, keeping 137319 word types
2021-02-23 20:07:16,161 : INFO : PROGRESS: at sentence #70000, processed 23293819 words, keeping 149541 word types
2021-02-23 20:07:16,684 : INFO

2021-02-23 20:07:45,288 : INFO : PROGRESS: at sentence #710000, processed 129482719 words, keeping 690729 word types
2021-02-23 20:07:45,796 : INFO : PROGRESS: at sentence #720000, processed 131326251 words, keeping 698923 word types
2021-02-23 20:07:46,413 : INFO : PROGRESS: at sentence #730000, processed 133405061 words, keeping 707962 word types
2021-02-23 20:07:49,916 : INFO : PROGRESS: at sentence #740000, processed 146962855 words, keeping 765561 word types
2021-02-23 20:07:54,060 : INFO : PROGRESS: at sentence #750000, processed 162247244 words, keeping 810963 word types
2021-02-23 20:07:58,535 : INFO : PROGRESS: at sentence #760000, processed 178597370 words, keeping 851152 word types
2021-02-23 20:07:59,403 : INFO : PROGRESS: at sentence #770000, processed 181862498 words, keeping 853644 word types
2021-02-23 20:08:00,287 : INFO : PROGRESS: at sentence #780000, processed 185139730 words, keeping 854579 word types
2021-02-23 20:08:00,758 : INFO : PROGRESS: at sentence #790000, 

2021-02-23 20:08:45,859 : INFO : PROGRESS: at sentence #1410000, processed 352027794 words, keeping 1196703 word types
2021-02-23 20:08:49,340 : INFO : PROGRESS: at sentence #1420000, processed 365689697 words, keeping 1223064 word types
2021-02-23 20:08:50,136 : INFO : PROGRESS: at sentence #1430000, processed 368701155 words, keeping 1223883 word types
2021-02-23 20:08:50,321 : INFO : PROGRESS: at sentence #1440000, processed 369401663 words, keeping 1224097 word types
2021-02-23 20:08:50,409 : INFO : PROGRESS: at sentence #1450000, processed 369708850 words, keeping 1224238 word types
2021-02-23 20:08:50,492 : INFO : PROGRESS: at sentence #1460000, processed 370007394 words, keeping 1224371 word types
2021-02-23 20:08:50,574 : INFO : PROGRESS: at sentence #1470000, processed 370318348 words, keeping 1224472 word types
2021-02-23 20:08:50,662 : INFO : PROGRESS: at sentence #1480000, processed 370650534 words, keeping 1224658 word types
2021-02-23 20:08:51,410 : INFO : PROGRESS: at se

2021-02-23 20:09:36,486 : INFO : Loading a fresh vocabulary
2021-02-23 20:09:38,109 : INFO : effective_min_count=5 retains 416342 unique words (27% of original 1492743, drops 1076401)
2021-02-23 20:09:38,110 : INFO : effective_min_count=5 leaves 533154263 word corpus (99% of original 534796798, drops 1642535)
2021-02-23 20:09:39,344 : INFO : deleting the raw counts dictionary of 1492743 items
2021-02-23 20:09:39,391 : INFO : sample=0.001 downsamples 35 most-common words
2021-02-23 20:09:39,392 : INFO : downsampling leaves estimated 476070309 word corpus (89.3% of prior 533154263)
2021-02-23 20:09:40,739 : INFO : estimated required memory for 416342 words and 300 dimensions: 1207391800 bytes
2021-02-23 20:09:40,739 : INFO : resetting layer weights
2021-02-23 20:11:05,226 : INFO : training model with 16 workers on 416342 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2021-02-23 20:11:06,243 : INFO : EPOCH 1 - PROGRESS: at 0.08% examples, 752630 words/s, in

2021-02-23 20:12:10,859 : INFO : EPOCH 1 - PROGRESS: at 16.06% examples, 1046634 words/s, in_qsize 31, out_qsize 0
2021-02-23 20:12:11,865 : INFO : EPOCH 1 - PROGRESS: at 16.41% examples, 1046383 words/s, in_qsize 31, out_qsize 0
2021-02-23 20:12:12,870 : INFO : EPOCH 1 - PROGRESS: at 16.75% examples, 1045505 words/s, in_qsize 31, out_qsize 0
2021-02-23 20:12:13,873 : INFO : EPOCH 1 - PROGRESS: at 17.11% examples, 1045313 words/s, in_qsize 31, out_qsize 0
2021-02-23 20:12:14,889 : INFO : EPOCH 1 - PROGRESS: at 17.46% examples, 1045360 words/s, in_qsize 32, out_qsize 0
2021-02-23 20:12:15,890 : INFO : EPOCH 1 - PROGRESS: at 17.80% examples, 1044711 words/s, in_qsize 31, out_qsize 0
2021-02-23 20:12:16,903 : INFO : EPOCH 1 - PROGRESS: at 18.13% examples, 1044035 words/s, in_qsize 29, out_qsize 2
2021-02-23 20:12:17,905 : INFO : EPOCH 1 - PROGRESS: at 18.49% examples, 1043782 words/s, in_qsize 30, out_qsize 1
2021-02-23 20:12:18,925 : INFO : EPOCH 1 - PROGRESS: at 18.82% examples, 1042435

...

2021-02-23 20:44:00,236 : INFO : EPOCH 5 - PROGRESS: at 45.07% examples, 1050272 words/s, in_qsize 31, out_qsize 0
2021-02-23 20:44:01,241 : INFO : EPOCH 5 - PROGRESS: at 45.19% examples, 1050684 words/s, in_qsize 30, out_qsize 2
2021-02-23 20:44:02,243 : INFO : EPOCH 5 - PROGRESS: at 45.32% examples, 1050789 words/s, in_qsize 31, out_qsize 3
2021-02-23 20:44:03,244 : INFO : EPOCH 5 - PROGRESS: at 45.46% examples, 1051398 words/s, in_qsize 32, out_qsize 0
2021-02-23 20:44:04,247 : INFO : EPOCH 5 - PROGRESS: at 45.59% examples, 1051538 words/s, in_qsize 31, out_qsize 0
2021-02-23 20:44:05,247 : INFO : EPOCH 5 - PROGRESS: at 45.71% examples, 1051490 words/s, in_qsize 31, out_qsize 0
2021-02-23 20:44:06,266 : INFO : EPOCH 5 - PROGRESS: at 45.83% examples, 1051721 words/s, in_qsize 31, out_qsize 0
2021-02-23 20:44:07,291 : INFO : EPOCH 5 - PROGRESS: at 45.97% examples, 1052118 words/s, in_qsize 31, out_qsize 0
2021-02-23 20:44:08,307 : INFO : EPOCH 5 - PROGRESS: at 46.10% examples, 1052445

2021-02-23 20:45:12,948 : INFO : EPOCH 5 - PROGRESS: at 55.20% examples, 1061756 words/s, in_qsize 30, out_qsize 1
2021-02-23 20:45:13,957 : INFO : EPOCH 5 - PROGRESS: at 55.50% examples, 1062005 words/s, in_qsize 31, out_qsize 0
2021-02-23 20:45:14,972 : INFO : EPOCH 5 - PROGRESS: at 55.76% examples, 1061597 words/s, in_qsize 31, out_qsize 0
2021-02-23 20:45:15,977 : INFO : EPOCH 5 - PROGRESS: at 56.03% examples, 1061531 words/s, in_qsize 31, out_qsize 0
2021-02-23 20:45:16,980 : INFO : EPOCH 5 - PROGRESS: at 56.30% examples, 1061559 words/s, in_qsize 30, out_qsize 1
2021-02-23 20:45:17,985 : INFO : EPOCH 5 - PROGRESS: at 56.56% examples, 1061528 words/s, in_qsize 30, out_qsize 0
2021-02-23 20:45:18,992 : INFO : EPOCH 5 - PROGRESS: at 56.81% examples, 1061388 words/s, in_qsize 31, out_qsize 1
2021-02-23 20:45:20,010 : INFO : EPOCH 5 - PROGRESS: at 57.06% examples, 1061254 words/s, in_qsize 31, out_qsize 0
2021-02-23 20:45:21,039 : INFO : EPOCH 5 - PROGRESS: at 57.31% examples, 1061081

2021-02-23 20:46:25,865 : INFO : EPOCH 5 - PROGRESS: at 72.91% examples, 1058850 words/s, in_qsize 31, out_qsize 0
2021-02-23 20:46:26,882 : INFO : EPOCH 5 - PROGRESS: at 73.02% examples, 1058841 words/s, in_qsize 31, out_qsize 0
2021-02-23 20:46:27,895 : INFO : EPOCH 5 - PROGRESS: at 73.15% examples, 1059102 words/s, in_qsize 31, out_qsize 0
2021-02-23 20:46:28,901 : INFO : EPOCH 5 - PROGRESS: at 73.27% examples, 1059060 words/s, in_qsize 30, out_qsize 1
2021-02-23 20:46:29,919 : INFO : EPOCH 5 - PROGRESS: at 73.39% examples, 1058870 words/s, in_qsize 22, out_qsize 9
2021-02-23 20:46:30,922 : INFO : EPOCH 5 - PROGRESS: at 73.53% examples, 1059082 words/s, in_qsize 32, out_qsize 1
2021-02-23 20:46:31,925 : INFO : EPOCH 5 - PROGRESS: at 73.66% examples, 1059199 words/s, in_qsize 30, out_qsize 1
2021-02-23 20:46:32,934 : INFO : EPOCH 5 - PROGRESS: at 73.79% examples, 1059356 words/s, in_qsize 31, out_qsize 0
2021-02-23 20:46:33,944 : INFO : EPOCH 5 - PROGRESS: at 73.91% examples, 1059512

2021-02-23 20:47:38,573 : INFO : EPOCH 5 - PROGRESS: at 88.87% examples, 1059846 words/s, in_qsize 32, out_qsize 0
2021-02-23 20:47:39,601 : INFO : EPOCH 5 - PROGRESS: at 89.22% examples, 1059640 words/s, in_qsize 31, out_qsize 0
2021-02-23 20:47:40,605 : INFO : EPOCH 5 - PROGRESS: at 89.58% examples, 1059722 words/s, in_qsize 31, out_qsize 0
2021-02-23 20:47:41,639 : INFO : EPOCH 5 - PROGRESS: at 90.00% examples, 1059565 words/s, in_qsize 31, out_qsize 2
2021-02-23 20:47:42,684 : INFO : EPOCH 5 - PROGRESS: at 90.47% examples, 1059386 words/s, in_qsize 31, out_qsize 0
2021-02-23 20:47:43,696 : INFO : EPOCH 5 - PROGRESS: at 90.99% examples, 1059321 words/s, in_qsize 31, out_qsize 0
2021-02-23 20:47:44,705 : INFO : EPOCH 5 - PROGRESS: at 91.50% examples, 1059265 words/s, in_qsize 32, out_qsize 1
2021-02-23 20:47:45,712 : INFO : EPOCH 5 - PROGRESS: at 91.99% examples, 1059192 words/s, in_qsize 31, out_qsize 0
2021-02-23 20:47:46,724 : INFO : EPOCH 5 - PROGRESS: at 92.47% examples, 1059034

[('discomfort', 0.8041051626205444),
 ('pain-', 0.7034913897514343),
 ('apin', 0.6237826347351074),
 ('pian', 0.6045260429382324),
 ('soreness', 0.5998637676239014),
 ('pain/', 0.5868990421295166),
 ('cramping', 0.5682056546211243),
 ('pain\n', 0.5675026774406433),
 ('discomfort-', 0.5263482928276062),
 ('ache', 0.5159716606140137)]

first run - using text tokenised and cleaned using nltk:

2021-02-22 11:04:46,937 : INFO : collected 2019963 word types from a corpus of 2083112 raw words and 2083112 sentences
2021-02-22 11:04:46,938 : INFO : Loading a fresh vocabulary
2021-02-22 11:04:47,731 : INFO : effective_min_count=5 retains 1176 unique words (0% of original 2019963, drops 2018787)
2021-02-22 11:04:47,732 : INFO : effective_min_count=5 leaves 23371 word corpus (1% of original 2083112, drops 2059741)
2021-02-22 11:04:47,736 : INFO : deleting the raw counts dictionary of 2019963 items
2021-02-22 11:04:47,993 : INFO : sample=0.001 downsamples 38 most-common words
2021-02-22 11:04:47,994 : INFO : downsampling leaves estimated 15132 word corpus (64.7% of prior 23371)
2021-02-22 11:04:47,997 : INFO : estimated required memory for 1176 words and 300 dimensions: 3410400 bytes
2021-02-22 11:04:47,998 : INFO : resetting layer weights
2021-02-22 11:04:48,240 : INFO : training model with 16 workers on 1176 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2021-02-22 11:05:06,369 : INFO : EPOCH - 1 : training on 2083112 raw words (15121 effective words) took 18.1s, 834 effective words/s
2021-02-22 11:05:24,363 : INFO : EPOCH - 2 : training on 2083112 raw words (15131 effective words) took 18.0s, 841 effective words/s
2021-02-22 11:05:42,383 : INFO : EPOCH - 3 : training on 2083112 raw words (15072 effective words) took 18.0s, 837 effective words/s
2021-02-22 11:05:59,885 : INFO : EPOCH - 4 : training on 2083112 raw words (15144 effective words) took 17.5s, 866 effective words/s
2021-02-22 11:06:17,620 : INFO : EPOCH - 5 : training on 2083112 raw words (15152 effective words) took 17.7s, 855 effective words/s
2021-02-22 11:06:17,621 : INFO : training on a 10415560 raw words (75620 effective words) took 89.4s, 846 effective words/s
2021-02-22 11:06:17,622 : INFO : saving Word2Vec object under ./noteevents_clean_fasttext_win10_min5.dat, separately None
2021-02-22 11:06:17,623 : INFO : not storing attribute vectors_norm
2021-02-22 11:06:17,624 : INFO : not storing attribute cum_table
2021-02-22 11:06:17,641 : INFO : saved ./noteevents_clean_fasttext_win10_min5.dat
2021-02-22 11:06:17,642 : INFO : precomputing L2-norms of word weight vectors

second run - using text cleaned with spacy and this script

2021-02-23 20:09:36,485 : INFO : collected 1492743 word types from a corpus of 534796798 raw words and 2083180 sentences
2021-02-23 20:09:36,486 : INFO : Loading a fresh vocabulary
2021-02-23 20:09:38,109 : INFO : effective_min_count=5 retains 416342 unique words (27% of original 1492743, drops 1076401)
2021-02-23 20:09:38,110 : INFO : effective_min_count=5 leaves 533154263 word corpus (99% of original 534796798, drops 1642535)
2021-02-23 20:09:39,344 : INFO : deleting the raw counts dictionary of 1492743 items
2021-02-23 20:09:39,391 : INFO : sample=0.001 downsamples 35 most-common words
2021-02-23 20:09:39,392 : INFO : downsampling leaves estimated 476070309 word corpus (89.3% of prior 533154263)
2021-02-23 20:09:40,739 : INFO : estimated required memory for 416342 words and 300 dimensions: 1207391800 bytes
2021-02-23 20:09:40,739 : INFO : resetting layer weights
2021-02-23 20:11:05,226 : INFO : training model with 16 workers on 416342 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2021-02-23 20:18:38,538 : INFO : EPOCH - 1 : training on 534796798 raw words (476062716 effective words) took 453.3s, 1050207 effective words/s
2021-02-23 20:26:04,708 : INFO : EPOCH - 2 : training on 534796798 raw words (476069483 effective words) took 446.2s, 1067028 effective words/s
2021-02-23 20:33:31,278 : INFO : EPOCH - 3 : training on 534796798 raw words (476083085 effective words) took 446.6s, 1066103 effective words/s
2021-02-23 20:41:00,405 : INFO : EPOCH - 4 : training on 534796798 raw words (476073531 effective words) took 449.1s, 1060016 effective words/s
2021-02-23 20:48:29,504 : INFO : EPOCH - 5 : training on 534796798 raw words (476069393 effective words) took 449.1s, 1060066 effective words/s
2021-02-23 20:48:29,506 : INFO : training on a 2673983990 raw words (2380358208 effective words) took 2244.3s, 1060634 effective words/s
2021-02-23 20:48:29,507 : INFO : saving Word2Vec object under ./notes_clean_w2v_win10_min5_v2.dat, separately None
2021-02-23 20:48:29,508 : INFO : storing np array 'vectors' to ./notes_clean_w2v_win10_min5_v2.dat.wv.vectors.npy
2021-02-23 20:48:29,855 : INFO : not storing attribute vectors_norm
2021-02-23 20:48:29,856 : INFO : storing np array 'syn1neg' to ./notes_clean_w2v_win10_min5_v2.dat.trainables.syn1neg.npy
2021-02-23 20:48:30,195 : INFO : not storing attribute cum_table
2021-02-23 20:48:31,003 : INFO : saved ./notes_clean_w2v_win10_min5_v2.dat
2021-02-23 20:48:31,004 : INFO : precomputing L2-norms of word weight vectors

In [42]:
# test
w2v_model.wv.most_similar("pain")

[('discomfort', 0.8041051626205444),
 ('pain-', 0.7034913897514343),
 ('apin', 0.6237826347351074),
 ('pian', 0.6045260429382324),
 ('soreness', 0.5998637676239014),
 ('pain/', 0.5868990421295166),
 ('cramping', 0.5682056546211243),
 ('pain\n', 0.5675026774406433),
 ('discomfort-', 0.5263482928276062),
 ('ache', 0.5159716606140137)]

## TRAIN TFIDF

In [ ]:
# TRAIN AND SAVE TFIDF (USING SKLEARN - DOESN'T SEEM TO WORK ON BIG DATASETS)
tfidf = TfidfVectorizer(min_df=min_df, max_df=max_df, ngram_range=ngram_range, sublinear_tf=True, use_idf=True, 
                        max_features=max_features, preprocessor=' '.join)
trained_tfidf = tfidf.fit(sentences)
with open(filename.replace('.txt','_tfidf.pickle'), 'wb') as fin:
          pickle.dump(trained_tfidf, fin)

In [ ]:
# TRAIN AND SAVE TFIDF (UING GENSIM)

# Create the Dictionary and Corpus
mydict = corpora.Dictionary(sentences) # create dctionary
mydict.save(filename.replace('.txt','_tfidfdict.dict'))  # save dict to disk
corpus = [mydict.doc2bow(line) for line in sentences] # create corpus
corpora.MmCorpus.serialize(filename.replace('.txt','_tfidfcorpus.mm'), corpus)  # save corpus to disk

tfidf_model = TfidfModel(corpus)  # fit model
tfidf_model.save(filename.replace('.txt','_tfidf.model')) # save model